<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Newton_Method_for_Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing required libraries
!pip install torchinfo
import torchinfo
import torch
from torch import nn,optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"

In [3]:
# Creating our own LeNet5

class LeNet5(nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5) # in channel , out channe, kernel
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d((2,2))

    self.conv2 = nn.Conv2d(6,16,5)  # in channel , out channe, kernel
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d((2,2))

    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = self.conv1(x)

    x = self.relu1(x)
    x= self.maxpool1(x)

    x = self.conv2(x)
    x = self.relu2(x)
    x= self.maxpool2(x)

    x = x.view(-1,int(x.nelement() / x.shape[0]))

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)



    return x



model = LeNet5().to(device)
torch.nn.init.xavier_uniform_(model.conv1.weight)
torch.nn.init.xavier_uniform_(model.conv2.weight)
torch.nn.init.xavier_uniform_(model.fc1.weight)
torch.nn.init.xavier_uniform_(model.fc2.weight)
torch.nn.init.xavier_uniform_(model.fc3.weight)



Parameter containing:
tensor([[-1.0485e-01,  2.6221e-02,  1.4305e-01,  1.1258e-01,  2.5070e-01,
         -1.0132e-01,  6.2419e-02, -2.3740e-01,  1.6299e-01, -1.3893e-01,
         -1.8354e-01,  2.0188e-01,  1.1389e-01, -1.1026e-01,  1.3351e-02,
         -7.2104e-02, -1.5396e-01, -4.9009e-02,  1.7185e-01, -1.3751e-01,
          6.3576e-02, -9.5254e-02, -1.9802e-01, -1.7262e-01,  1.0346e-01,
         -1.4084e-01, -1.1738e-01,  5.1683e-02,  1.4973e-01,  7.8608e-02,
          9.5203e-02,  1.8488e-01, -8.1277e-02,  1.0353e-01,  9.4987e-02,
         -1.1507e-01,  1.5491e-01, -1.8417e-01, -6.1050e-02,  2.4740e-01,
         -2.0524e-01, -1.3729e-01, -7.1980e-03, -1.1359e-01,  1.6201e-01,
          1.5694e-01,  1.7936e-01, -1.2450e-01, -8.3718e-02,  1.4556e-01,
          1.6879e-01, -1.7526e-01,  2.3363e-01,  2.3453e-01,  1.4639e-01,
          2.0151e-01, -1.2142e-02,  6.9319e-02,  8.4938e-02,  2.2199e-02,
         -1.6115e-01, -5.9874e-02,  9.5377e-02,  8.9639e-02, -1.1257e-02,
          1.8728

In [4]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
train_transforms = transforms.Compose([ transforms.ToTensor(), transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])
train_data = CIFAR10(root="./train/", train=False, download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader( train_data, shuffle=True)

100%|██████████| 170498071/170498071 [00:03<00:00, 44309347.30it/s]


Extracting ./train/cifar-10-python.tar.gz to ./train/


In [37]:
#from torch.optim import newton_cg
import torchmin
# from torchmin import minimize
criterion = nn.CrossEntropyLoss()
optimizer = torchmin.Minimizer(model.parameters(),method='l-bfgs')


In [38]:
N_EPOCHS = 10
for epoch in range(N_EPOCHS):
  epoch_loss = 0.0
  for inputs, labels in trainloader:
    inputs = inputs.to(device)
    labels = labels.to(device)


    def closure():
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward(retain_graph=True)
      torch.nn.utils.clip_grad_norm_(model.parameters(),1.0e-2)

      return loss
    optimizer.step(closure)
    loss = closure()
    epoch_loss += loss.item()

  print("Epoch: {} Loss: {}".format(epoch,epoch_loss/len(trainloader)))



KeyboardInterrupt: ignored